<a href="https://colab.research.google.com/github/Ygrick/colab/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pendulum

     |████████████████████████████████| 163kB 2.9MB/s 
     |████████████████████████████████| 491kB 15.3MB/s 


In [ ]:
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy
import requests
import pandas as pd
from matplotlib import pyplot as plt
import datetime
from datetime import date


def getRcode(message):
    df = pd.read_csv('file1.csv', delimiter=',')
    df.drop('Unnamed: 0', axis=1)
    dfName = df.set_index('Name')
    dfEngName = df.set_index('EngName')
    try:
        return str(dfName.loc[message, 'Rcode'])[:6]
    except KeyError:
        try:
            return str(dfEngName.loc[message, 'Rcode'])[:6]
        except KeyError:
            return "Нет такой валюты"


def get_period(fromDate, toDate, Rcode):
    html = requests.get(f'https://cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={Rcode}&UniDbQuery.From={fromDate}&UniDbQuery.To={toDate}').text
    table = ((pd.concat(pd.read_html(html), ignore_index = True)).drop(index=[0,1])).reset_index(drop=True)
    table.columns = ['date', 'multiplier', 'rate']
    table['rate'], table['multiplier'] = pd.to_numeric(table['rate'],downcast = "float"), pd.to_numeric(table['multiplier'])
    table['date'] = pd.to_datetime(table['date'],format="%d.%m.%Y")
    for n in range(table.shape[0]):
        table['rate'][n] /= table['multiplier'][n]*10000
    table = table.drop('multiplier', axis=1)
    table = table.iloc[::-1].reset_index(drop=True)
    return table


def get_prediction(curr):
    today = datetime.datetime.today()
    tomorrow = (today + datetime.timedelta(days=1))
    df = get_period('01.01.2019',today, getRcode(curr)) # с 2016 по текущее с Rcode нужным
    new_row = {'date': [tomorrow], 'rate': [0]}
    new_df = pd.DataFrame(new_row)
    df = pd.concat([df, new_df])
    
    # df['date'] = pd.to_datetime(df['date']).dt.date
    # df['date'] = pd.to_datetime(df['date'],format="%d.%m.%Y")
    # разбиение даты на года\месяцы\недели
    df["weekday"] = df["date"].dt.weekday
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    df.drop(['date'],axis=1,inplace=True)
    
    # курсы за последние 7 дней (скроллинг\шифтинг)
    past_days = 7
    for day in range(past_days):
      d = day+1
      df[f"curs_back_{d}d"] = df["rate"].shift(d)
    df.dropna(inplace = True)
    
    # бинарность столбцов по значениям
    df = pd.get_dummies(df, columns=["year", "month", "weekday"])
    print(df)
    # нужно создать последннюю строчку(как раз которую будем прогнозировать)
    # а именно входные данные
    new_df = df[-1:]
    df.drop(df.tail(1).index,inplace=True)
    # разметка на тестовые и тренировочные данные
    x = df.drop('rate',axis=1)
    y = df['rate']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
    
    # обучение 
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # метрика
    prediction = model.predict(X_test)
    print(mean_absolute_error(y_test,prediction))
    print(max_error(y_test,prediction))
    
    # предсказание
    return model.predict(new_df.drop('rate',axis=1))


print(get_prediction('Японская иена'))




         rate  curs_back_1d  curs_back_2d  ...  weekday_4  weekday_5  weekday_6
7    0.606066      0.610893      0.614834  ...          0          1          0
8    0.605257      0.606066      0.610893  ...          0          0          0
9    0.608345      0.605257      0.606066  ...          0          0          0
10   0.605300      0.608345      0.605257  ...          0          0          0
11   0.601847      0.605300      0.608345  ...          1          0          0
..        ...           ...           ...  ...        ...        ...        ...
582  0.674877      0.675920      0.677192  ...          0          0          0
583  0.675031      0.674877      0.675920  ...          0          0          0
584  0.673525      0.675031      0.674877  ...          1          0          0
585  0.669917      0.673525      0.675031  ...          0          1          0
0    0.000000      0.669917      0.673525  ...          0          0          0

[580 rows x 30 columns]
0.0046450807534

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
